In [12]:
import pandas as pd
import numpy as np
import os


In [13]:
data_dir = r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Files"
merged_file_path=r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Ranked Stocks"

files = [
    "PE_Annual.csv",
    "PB_Annual.csv",
    "DividendYield_Data.csv",
    "MarketCap_Data.csv",
    "ROE_Annual.csv",
    "ROA_Annual.csv",
    "Volume_data.csv",
    "DebtToEquity_Annual.csv"
]


First we are merging all the files of parameters together into one single csv file 

In [14]:

dfs = []

for filename in files:
    filepath = os.path.join(data_dir, filename)
    df = pd.read_csv(filepath)
    
    df.columns = df.columns.str.strip()

    # Ensure Ticker exists
    if 'Ticker' not in df.columns:
        raise KeyError(f"'Ticker' column missing in {filename}")
    
    # Normalize Year
    if 'Year' in df.columns:
        df['Year'] = df['Year'].astype(int)
    elif 'Date' in df.columns:
        df['Year'] = pd.to_datetime(df['Date'], errors='coerce').dt.year
        df.drop(columns=['Date'], inplace=True)
    else:
        raise KeyError(f"'{filename}' must have either 'Date' or 'Year' column.")

    df['Ticker'] = df['Ticker'].astype(str).str.strip()
    
    dfs.append(df)


merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on=['Ticker', 'Year'], how='outer')

output_path = os.path.join(merged_file_path, "final_merged_by_year.csv")
merged_df.to_csv(output_path, index=False)
print(f"Merged file saved to: {output_path}")

print("Final merged columns:", merged_df.columns.tolist())
print(merged_df[['Ticker', 'Year']].head())


Merged file saved to: C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Ranked Stocks\final_merged_by_year.csv
Final merged columns: ['Ticker', 'StockPrice_x', 'NetEPS', 'PE', 'Year', 'StockPrice_y', 'BookValuePerShare', 'PB', 'DividendYield', 'MarketCap', 'ROE', 'NetIncome', 'ROA', 'TotalAssets', 'Volume', 'DebtToEquity']
  Ticker  Year
0   AAPL  2010
1   AAPL  2011
2   AAPL  2012
3   AAPL  2013
4   AAPL  2014


In [16]:
def calculate_rolling_percentile(df, col, window=5):
    return df.groupby('Ticker')[col].transform(
        lambda x: x.rolling(window=window, min_periods=1).apply(
            lambda s: pd.Series.rank(s).iloc[-1] / len(s)
        )
    )

# Calculate rolling percentiles
merged_df['PE_percentile'] = 1 - calculate_rolling_percentile(merged_df, 'PE')
merged_df['PB_percentile'] = 1 - calculate_rolling_percentile(merged_df, 'PB')
merged_df['ROE_percentile'] = calculate_rolling_percentile(merged_df, 'ROE')
merged_df['ROA_percentile'] = calculate_rolling_percentile(merged_df, 'ROA')
merged_df['DebtToEquity_percentile'] = 1 - calculate_rolling_percentile(merged_df, 'DebtToEquity')

# Combine into category scores
merged_df['Value_Score'] = (merged_df['PE_percentile'] + merged_df['PB_percentile']) / 2
merged_df['Quality_Score'] = (merged_df['ROE_percentile'] + merged_df['ROA_percentile']) / 2
merged_df['Risk_Score'] = merged_df['DebtToEquity_percentile']

# Total weighted score
merged_df['Total_Score'] = (
    0.4 * merged_df['Value_Score'] +
    0.4 * merged_df['Quality_Score'] +
    0.2 * merged_df['Risk_Score']
)

# Make sure 'Year' column exists
if 'Year' not in merged_df.columns:
    if 'Date' in merged_df.columns:
        merged_df['Year'] = pd.to_datetime(merged_df['Date'], errors='coerce').dt.year
    else:
        raise ValueError("No 'Year' or 'Date' column to extract Year from.")

# Filter stepwise by Year (example: keep top 150 by ROE, then top 100 by ROA, etc.)
def stepwise_filtering(df, year, filters):
    yearly_df = df[df['Year'] == year].copy()
    for col, ascending, n in filters:
        yearly_df = yearly_df.sort_values(col, ascending=ascending).head(n)
    return yearly_df

# Define your filtering sequence (example)
filters = [
    ('ROE', False, 150),       # highest ROE keep top 150
    ('ROA', False, 100),       # highest ROA keep top 100
    ('DividendYield', False, 80), # highest DividendYield keep top 80
    ('DebtToEquity', True, 60),   # lowest DebtToEquity keep top 60
    ('PE', True, 50),          # lowest PE keep top 50
    ('PB', True, 40),          # lowest PB keep top 40
    ('MarketCap', False, 30),  # highest MarketCap keep top 30
    ('Volume', False, 20)      # highest Volume keep top 20
]

# Run filtering & collect filtered data for each year
filtered_list = []
for year in merged_df['Year'].unique():
    filtered_year_df = stepwise_filtering(merged_df, year, filters)
    filtered_list.append(filtered_year_df)

filtered_df = pd.concat(filtered_list)

# Now rank filtered stocks by Total_Score per year
filtered_df['Rank'] = filtered_df.groupby('Year')['Total_Score'].rank(ascending=False, method='dense')

# Save final ranked stocks per year
final_cols = ['Ticker', 'Year', 'PE', 'PB', 'ROE', 'ROA', 'DebtToEquity',
              'DividendYield', 'MarketCap', 'Volume',
              'Value_Score', 'Quality_Score', 'Risk_Score', 'Total_Score', 'Rank']

output_path = r"C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Ranked Stocks\final_scored_ranked.csv"
filtered_df[final_cols].to_csv(output_path, index=False)
print(f"✅ Final ranked file saved to: {output_path}")



✅ Final ranked file saved to: C:\Users\surji\Desktop\Quant_Poject\Complete_analysis_200stocks\Data\Ranked Stocks\final_scored_ranked.csv
